# Pandas

For SQL and Pandas, if the aggregation results are incorrect and you are very sure of the logic. Often times we have forgotten to check for duplicates for the selected rows before aggregation.

## Implement dataframe using dict and list, and types

In [1]:
import pandas as pd
import numpy as np

df1 = pd.DataFrame(
    {
        "a": [2, 3, 4, 5],
        "b": ["a", "b", "c", "d"],
        "c": [[1, 2], [3, 4], [5, 6], [7, 8]],
    },
    # index=[0, 1, 2, 3]
)

df1

,a,b,c
0,2,a,"[1, 2]"
1,3,b,"[3, 4]"
2,4,c,"[5, 6]"
3,5,d,"[7, 8]"


In [2]:
df2 = pd.DataFrame(
    [
        # different from dict implementation
        [2, "a", [1, 2]],
        [3, "b", [3, 4]],
        [4, "c", [5, 6]],
        [5, "d", [7, 8]],
    ],
    columns=["a", "b", "c"],
)

df2

,a,b,c
0,2,a,"[1, 2]"
1,3,b,"[3, 4]"
2,4,c,"[5, 6]"
3,5,d,"[7, 8]"


In [3]:
print(df2.dtypes)

df2 = df2.astype({"a": float})

df2["a"] = df2["a"].apply(pd.to_numeric)
df2[["a",]] = df2[["a",]].apply(pd.to_numeric) # if multiple columns to apply function

a     int64
b    object
c    object
dtype: object


## Concat Series or dataframe

In [4]:
# Combine 2 similar df rowwise
pd.concat([df1, df2], axis=0)

,a,b,c
0,2.0,a,"[1, 2]"
1,3.0,b,"[3, 4]"
2,4.0,c,"[5, 6]"
3,5.0,d,"[7, 8]"
0,2.0,a,"[1, 2]"
1,3.0,b,"[3, 4]"
2,4.0,c,"[5, 6]"
3,5.0,d,"[7, 8]"


In [5]:
pd.concat([df1["a"], df2["a"]], axis=1)  # works on Series too

# Combine 2 similar df colwise (*based on index)
pd.concat([df1, df2], axis=1)

,a,b,c,a,b,c
0,2,a,"[1, 2]",2.0,a,"[1, 2]"
1,3,b,"[3, 4]",3.0,b,"[3, 4]"
2,4,c,"[5, 6]",4.0,c,"[5, 6]"
3,5,d,"[7, 8]",5.0,d,"[7, 8]"


## Melt and Pivot Transformations

In [6]:
data = {
    'ID': [1, 2, 3],
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Math_Score': [90, 85, 78],
    'English_Score': [88, 92, 80],
    'History_Score': [75, 87, 89]
}

df = pd.DataFrame(data)

df

,ID,Name,Math_Score,English_Score,History_Score
0,1,Alice,90,88,75
1,2,Bob,85,92,87
2,3,Charlie,78,80,89


In [7]:
# WIDE TO LONG
melted_df = df.melt(
    id_vars = ['ID', 'Name'],
    value_vars = ['Math_Score','English_Score', 'History_Score'],
    var_name = "Subject",
    value_name = "Score"
)
melted_df

,ID,Name,Subject,Score
0,1,Alice,Math_Score,90
1,2,Bob,Math_Score,85
2,3,Charlie,Math_Score,78
3,1,Alice,English_Score,88
4,2,Bob,English_Score,92
5,3,Charlie,English_Score,80
6,1,Alice,History_Score,75
7,2,Bob,History_Score,87
8,3,Charlie,History_Score,89


In [8]:
# LONG TO WIDE
long_df = melted_df.pivot(
    index=["ID", "Name"],
    columns="Subject",
    values="Score"
).reset_index().rename_axis(None, axis=1)

long_df

,ID,Name,English_Score,History_Score,Math_Score
0,1,Alice,88,75,90
1,2,Bob,92,87,85
2,3,Charlie,80,89,78


## Piping functions (Sorting, renaming, dropping)

In [9]:
(
    df
    .sort_values(by=['English_Score', 'History_Score'], ascending=[False, True])
    .rename(columns = {'Name': 'Naming', 'ID': 'NRIC'})
    .drop(columns = ['NRIC'])
    .drop_duplicates()
    .reset_index(drop=True)
)

,Naming,Math_Score,English_Score,History_Score
0,Bob,85,92,87
1,Alice,90,88,75
2,Charlie,78,80,89


## Sampling Values

In [10]:
df.sample(1)  # Sample 1 row

df[["Name", "Math_Score"]]  # Subset df based on col names

df["Name"]  # Series object

df.iloc[1, :3]  # i := index, else use names

df.iat[1, 3]

92

In [11]:
df.nlargest(2, "Math_Score") 
# df.nsmallest(2, "Math_Score")
# df.head(2)
# df.tail(2)

,ID,Name,Math_Score,English_Score,History_Score
0,1,Alice,90,88,75
1,2,Bob,85,92,87


In [12]:
df.filter(like="Name")

,Name
0,Alice
1,Bob
2,Charlie


## Filtering Functions

In [13]:
df[
    # & | ^ (and/or/xor)
    (df["Name"] == "Bob")
    & (df["English_Score"] >= 90)
    & (~df["History_Score"].isna())  # ~ flips logic or use notna()
    & (df["English_Score"].isin([90, 91, 92]))
]

df.dropna()

df["Name"] = df["Name"].fillna("0")

In [14]:
class ListNode():
    def __init__(self, val=0, next=None):

        self.val = val
        self.next = next

a = ListNode()
b = ListNode(3,a)


In [15]:
(df["Math_Score"] > 80), (df["Math_Score"] > 80).any(), (df["Math_Score"] > 80).all()
# Check series

(0     True
 1     True
 2    False
 Name: Math_Score, dtype: bool,
 True,
 False)

## Descriptive Statistics


In [16]:
df["Name"].value_counts()

Name
Alice      1
Bob        1
Charlie    1
Name: count, dtype: int64

In [17]:
len(df),
df.shape,
df["Name"].nunique()

3

In [18]:
df.describe()

,ID,Math_Score,English_Score,History_Score
count,3.0,3.000000,3.000000,3.000000
mean,2.0,84.333333,86.666667,83.666667
std,1.0,6.027714,6.110101,7.571878
min,1.0,78.000000,80.000000,75.000000
25%,1.5,81.500000,84.000000,81.000000
50%,2.0,85.000000,88.000000,87.000000
75%,2.5,87.500000,90.000000,88.000000
max,3.0,90.000000,92.000000,89.000000


In [19]:
df["Math_Score"].mean()
# mean, median, count, sum, min, max, var, std, quantile([0.25,0.75])

84.33333333333333

In [20]:
# sum rowwise (add across row)
df.apply(np.sum, axis=0)


# Note for apply, if df['column'] series object, do not need specify axis, as it is single column

ID                             6
Name             AliceBobCharlie
Math_Score                   253
English_Score                260
History_Score                251
dtype: object

In [21]:
df[["Math_Score", "English_Score"]].apply(lambda x: x+30, axis=0)

,Math_Score,English_Score
0,120,118
1,115,122
2,108,110


In [22]:
df["English_desc"] = df.apply(lambda row: row["Name"] + " score is " + str(row["English_Score"]), axis=1)

df

,ID,Name,Math_Score,English_Score,History_Score,English_desc
0,1,Alice,90,88,75,Alice score is 88
1,2,Bob,85,92,87,Bob score is 92
2,3,Charlie,78,80,89,Charlie score is 80


## Making New Columns

In [23]:
df["Total_scores"] =  df["English_Score"] + df["History_Score"] + df["Math_Score"]

## Joins

In [24]:
data1 = {
    'ID': [1, 2, 3, 4],
    'Name': ['Alice', 'Bob', 'Charlie', 'David'],
    'Age': [25, 30, 22, 28]
}

df1 = pd.DataFrame(data1)

data2 = {
    'ID': [1, 2, 3, 5],
    'Name': ['Tan', 'Sim', 'Tsang', 'Lim'],
    'City': ['New York', 'San Francisco', 'Los Angeles', 'Chicago'],
    'Occupation': ['Engineer', 'Data Scientist', 'Artist', 'Teacher']
}

df2 = pd.DataFrame(data2)

In [25]:
merged_df = pd.merge(
    df1,
    df2,
    how="outer", # outer inner right left cross
    on="ID", # left_on='ID', right_on='ID'
    indicator=True,
    suffixes= ["_left", "_right"]
)

merged_df

,ID,Name_left,Age,Name_right,City,Occupation,_merge
0,1,Alice,25.0,Tan,New York,Engineer,both
1,2,Bob,30.0,Sim,San Francisco,Data Scientist,both
2,3,Charlie,22.0,Tsang,Los Angeles,Artist,both
3,4,David,28.0,NaN,NaN,NaN,left_only
4,5,NaN,NaN,Lim,Chicago,Teacher,right_only


## Group By Agg Functions

In [26]:
data = {
    'Date': pd.date_range(start='2022-01-01', end='2022-01-10', freq='D').tolist() * 3,
    'Product': ['A', 'B', 'C'] * 10,
    'Category': ['Electronics', 'Clothing', 'Home'] * 10,
    'Sales': np.random.randint(100, 1000, size=30),
    'Prices': np.random.randint(100, 1000, size=30),
    'Region': ['North', 'South', 'East', 'West'] * 7 + ['Central'] * 2
}

df = pd.DataFrame(data)
df.head()

,Date,Product,Category,Sales,Prices,Region
0,2022-01-01,A,Electronics,959,588,North
1,2022-01-02,B,Clothing,112,489,South
2,2022-01-03,C,Home,428,485,East
3,2022-01-04,A,Electronics,360,934,West
4,2022-01-05,B,Clothing,330,381,North


In [27]:
pd.merge(
    df.groupby('Category')['Sales'].sum().reset_index(), # without reset_index it is a series
    df.groupby('Category')['Prices'].sum().reset_index()
)

# TO group on multiple, input a list of columns

,Category,Sales,Prices
0,Clothing,5273,4981
1,Electronics,5063,5255
2,Home,5106,5735


In [28]:
df.groupby("Category")["Prices"].nlargest(2).reset_index().drop(columns=["level_1"])

,Category,Prices
0,Clothing,795
1,Clothing,775
2,Electronics,995
3,Electronics,934
4,Home,960
5,Home,936


In [29]:
df['Sales_Cumsum'] = df.groupby('Category')['Sales'].cumsum() # setting column to a series with matching indexes

df.head(5)

,Date,Product,Category,Sales,Prices,Region,Sales_Cumsum
0,2022-01-01,A,Electronics,959,588,North,959
1,2022-01-02,B,Clothing,112,489,South,112
2,2022-01-03,C,Home,428,485,East,428
3,2022-01-04,A,Electronics,360,934,West,1319
4,2022-01-05,B,Clothing,330,381,North,442


In [30]:
# IMPORTANT
df.groupby('Category')['Prices'].agg(["sum", "max"]).reset_index() # Reset the multi level index

,Category,sum,max
0,Clothing,4981,795
1,Electronics,5255,995
2,Home,5735,960


## Iterating rows & Col names

In [31]:
for index, row in df.iterrows():
    print(index, row["Category"], row["Sales"])

0 Electronics 959
1 Clothing 112
2 Home 428
3 Electronics 360
4 Clothing 330
5 Home 338
6 Electronics 577
7 Clothing 777
8 Home 711
9 Electronics 547
10 Clothing 622
11 Home 504
12 Electronics 772
13 Clothing 684
14 Home 504
15 Electronics 348
16 Clothing 274
17 Home 851
18 Electronics 172
19 Clothing 666
20 Home 818
21 Electronics 273
22 Clothing 811
23 Home 341
24 Electronics 808
25 Clothing 164
26 Home 237
27 Electronics 247
28 Clothing 833
29 Home 374


In [32]:
df.columns

Index(['Date', 'Product', 'Category', 'Sales', 'Prices', 'Region',
       'Sales_Cumsum'],
      dtype='object')

## Handling datetime

In [33]:
df = pd.DataFrame({
    "created_at": pd.date_range(start="2022-01-01", periods=100),
    "category": np.random.choice(['A', 'B', 'C'], size=100),
    "Value1": np.random.randint(1, 101, size=100),
    "Value2": np.random.normal(0, 1, 100),
})

In [34]:
df['year_month'] = df["created_at"].dt.strftime('%Y-%m')

df.head(3)

,created_at,category,Value1,Value2,year_month
0,2022-01-01,B,47,-0.223794,2022-01
1,2022-01-02,B,27,-1.645556,2022-01
2,2022-01-03,A,26,0.206114,2022-01


## Lag and Lead

In [35]:
df["Value1_lag"] = df['Value1'].shift(periods=1, fill_value=0)
df.head(3)

,created_at,category,Value1,Value2,year_month,Value1_lag
0,2022-01-01,B,47,-0.223794,2022-01,0
1,2022-01-02,B,27,-1.645556,2022-01,47
2,2022-01-03,A,26,0.206114,2022-01,27


In [36]:
df["Value2_lag_by_category"] = df.groupby('category')['Value2'].transform(lambda x: x.shift(periods=1, fill_value=0))
df.head(3)

,created_at,category,Value1,Value2,year_month,Value1_lag,Value2_lag_by_category
0,2022-01-01,B,47,-0.223794,2022-01,0,0.000000
1,2022-01-02,B,27,-1.645556,2022-01,47,-0.223794
2,2022-01-03,A,26,0.206114,2022-01,27,0.000000


## Standardizing

In [37]:
df["std_Value2"] = df.groupby('category')['Value2'].transform(lambda x: (x-x.mean())/x.std())
df.head(3)

,created_at,category,Value1,Value2,year_month,Value1_lag,Value2_lag_by_category,std_Value2
0,2022-01-01,B,47,-0.223794,2022-01,0,0.000000,-0.139833
1,2022-01-02,B,27,-1.645556,2022-01,47,-0.223794,-1.433669
2,2022-01-03,A,26,0.206114,2022-01,27,0.000000,0.486675


## Percentile & Ranking

In [38]:
df["percentile"] = df.groupby(['category'])['Value2'].rank(pct = True) # pct returns the percentile form

df[df["percentile"] > 0.95]

,created_at,category,Value1,Value2,year_month,Value1_lag,Value2_lag_by_category,std_Value2,percentile
28,2022-01-29,C,34,2.741430,2022-01,87,1.369396,2.279343,1.000000
29,2022-01-30,B,98,1.948840,2022-01,34,-1.117669,1.837313,1.000000
47,2022-02-17,A,31,2.442694,2022-02,76,-1.851729,2.926903,1.000000
51,2022-02-21,B,90,1.841580,2022-02,47,0.198765,1.739704,0.974359
89,2022-03-31,A,27,0.982400,2022-03,66,-0.804674,1.333645,0.967742
94,2022-04-05,C,31,2.371341,2022-04,94,-1.076970,1.951188,0.966667


In [39]:
# MySQL -- DENSE_RANK() does not skip NUMBER 1 1 2 3
# MySQL -- RANK() skips number e.g. 1 1 3 4
# Pandas -- dense
# Pandas -- min

df["ranking"] = df.groupby(['category'])['Value1'].rank(ascending=False, method='dense')

df[df["ranking"] >= 2].head(3)

,created_at,category,Value1,Value2,year_month,Value1_lag,Value2_lag_by_category,std_Value2,percentile,ranking
0,2022-01-01,B,47,-0.223794,2022-01,0,0.000000,-0.139833,0.435897,18.0
1,2022-01-02,B,27,-1.645556,2022-01,47,-0.223794,-1.433669,0.102564,25.0
2,2022-01-03,A,26,0.206114,2022-01,27,0.000000,0.486675,0.741935,20.0


## Practise

In [40]:
import pandas as pd
import numpy as np

data = {
    "Category": np.random.choice(["A", "B", "C"], size=100),
    "Date": pd.date_range(start="2022-01-01", periods=100),
    "Value1": np.random.randint(1, 101, size=100),
    "Value2": np.random.normal(0, 1, 100),
}

df = pd.DataFrame(data)

df.head(3)

,Category,Date,Value1,Value2
0,C,2022-01-01,97,1.048154
1,C,2022-01-02,35,-0.139401
2,C,2022-01-03,40,0.187155


In [41]:
# 1. Calculate the cumulative sum of 'Value1' for each category
sorted_df = df.sort_values(by=["Category", "Value1"], ascending=[True, False])
sorted_df["cum_sum_Value1"] = sorted_df.groupby("Category")["Value1"].cumsum()
answer_1 = sorted_df.reset_index(drop=True)

In [42]:
# 2. Select the top 3 rows for each category based on the highest sum of 'Value2'
df.groupby("Category")["Value2"].nlargest(3).reset_index().drop(columns=["level_1"])

answer_2 = pd.merge(
    df,
    df.groupby("Category")["Value2"]
    .nlargest(3)
    .reset_index()
    .drop(columns=["level_1"]),
    on=["Category", "Value2"],
    how="inner",
).sort_values(by=["Category", "Value2"])

In [43]:
# 3. Find the count of unique dates in each category
answer_3 = df.groupby("Category")["Date"].nunique()

In [44]:
# 4. Rename the columns to 'Category_ID', 'Transaction_Date', 'Total_Value', 'Random_Value'
answer_4 = df.rename(
    columns={
        "Category": "Category_ID",
        "Date": "Transaction_Date",
        "Value1": "Total_Value",
        "Value2": "Random_Value",
    }
)

In [45]:
# 5. Add a new column 'Value2_Rank' which represents the rank of each value within its category for 'Value2'
df["Value2_Rank"] = df.groupby("Category")["Value2"].rank(method='dense', ascending=False).astype('int')

In [46]:
# 6. Create a new DataFrame that contains only rows where the absolute value of 'Value2' is greater than 1
answer_6 = df[np.absolute(df['Value2']) > 1]